In [146]:
from pyspark.sql.functions import desc

In [147]:
# Paths to the data files
country_dir = "/Users/sheng_chen/Downloads/getFieldsCountPerCountry/countries/*.json"
fields_dir = "/Users/sheng_chen/Downloads/getFieldsCountPerCountry/fields/*.json"
grower_dir = "/Users/sheng_chen/Downloads/getFieldsCountPerCountry/growers/*.json"

In [148]:
# Read all data files and store as dataframes
countries = sqlContext.read.json(country_dir)
fields = sqlContext.read.json(fields_dir)
growers = sqlContext.read.json(grower_dir)

In [149]:
# Select the columns we are interested in
countries = countries.select('country.id','country.alpha3')
fields = fields.select('field.associations.`agrian.grower`').toDF('id')
growers = growers.select('grower.id','grower.address.country_id')

In [150]:
# (Here we assume if a field has multiple growers, the first grower's country is representing of the field)
# Join growers and fields by grower ID
s = fields.join(growers, growers.id==fields.id[0]).select('country_id')
# Join countries and result from previous step, group by country name, show counts, and sort by descending count.
s = countries.join(s, s.country_id==countries.id).groupby('alpha3').count().sort(desc("count")).toDF("Country","Count").show()

+-------+-----+
|Country|Count|
+-------+-----+
|    USA|    4|
| CANADA|    1|
+-------+-----+

